## 📖 Teoria: String Parsing

### strtok - Tokenizzazione

```c
char *strtok(char *str, const char *delim);
```

**Come funziona:**
```c
char input[] = "192.168.1.0/24";
               └─────┬─────┘ └┬┘
                  token1    delim

char *ip = strtok(input, "/");   // "192.168.1.0"
char *prefix = strtok(NULL, "/"); // "24"
                    ↑
                 NULL = continua da dove ha lasciato
```

**⚠️ IMPORTANTE:** `strtok` modifica la stringa originale!

```
Prima:  "192.168.1.0/24"
Dopo:   "192.168.1.0\024"  ← '/' sostituito con '\0'
```

## 📖 Teoria: Formati Input

### 1. CIDR Notation

```
192.168.1.0/24
└─────┬─────┘ └┬┘
   network   prefix_len

Parsing:
  1. Split su '/'
  2. Parse IP: string_to_ip("192.168.1.0")
  3. Parse prefix: atoi("24")
  4. Calcola netmask: create_netmask(24)
  5. Calcola network: ip & netmask
```

### 2. Range

```
10.0.0.1-10.0.0.50
└───┬───┘ └───┬───┘
   start     end

Parsing:
  1. Split su '-'
  2. Parse start: string_to_ip("10.0.0.1")
  3. Parse end: string_to_ip("10.0.0.50")
  4. Valida: start <= end
```

### 3. Single IP

```
8.8.8.8
└──┬──┘
  unico IP

Parsing:
  1. Nessun '/' o '-'
  2. Parse IP: string_to_ip("8.8.8.8")
```

## 📝 Implementazione

In [ ]:
%%bash
mkdir -p /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/06
cd /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/06

# Copia file precedenti
cp ../05/ip_types.h .
cp ../05/ip_utils.* .
cp ../05/ipset.* .

# Crea parser.h
cat > parser.h << 'EOF'
#ifndef PARSER_H
#define PARSER_H

#include <stdint.h>
#include <stdbool.h>
#include "ipset.h"

/**
 * Tipi di specifica IP
 */
typedef enum {
    SPEC_CIDR,    // 192.168.1.0/24
    SPEC_RANGE,   // 10.0.0.1-10.0.0.50
    SPEC_SINGLE   // 8.8.8.8
} SpecType;

/**
 * Specifica subnet/range parsed
 */
typedef struct {
    SpecType type;
    uint32_t start_ip;   // IP iniziale (o network per CIDR)
    uint32_t end_ip;     // IP finale (o broadcast per CIDR)
    uint8_t prefix_len;  // Solo per CIDR
} IPSpec;

// Parsing
bool parse_ip_spec(const char *input, IPSpec *spec);
bool parse_cidr(const char *input, IPSpec *spec);
bool parse_range(const char *input, IPSpec *spec);
bool parse_single(const char *input, IPSpec *spec);

// Espansione
bool expand_spec_to_set(const IPSpec *spec, IPSet *set);

#endif // PARSER_H
EOF

echo "✅ parser.h creato"

In [ ]:
%%bash
cd /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/06

cat > parser.c << 'EOF'
#include "parser.h"
#include "ip_utils.h"
#include <string.h>
#include <stdlib.h>
#include <stdio.h>

/**
 * Parse CIDR: 192.168.1.0/24
 * 
 * Algoritmo:
 * 1. Copia input (strtok modifica stringa!)
 * 2. Split su '/'
 * 3. Parse IP e prefix
 * 4. Calcola network e broadcast
 */
bool parse_cidr(const char *input, IPSpec *spec) {
    char buf[64];
    strncpy(buf, input, sizeof(buf) - 1);
    buf[sizeof(buf) - 1] = '\0';
    
    // Tokenize: ip/prefix
    char *ip_str = strtok(buf, "/");
    char *prefix_str = strtok(NULL, "/");
    
    if (!ip_str || !prefix_str) return false;
    
    // Parse IP
    uint32_t ip;
    if (!string_to_ip(ip_str, &ip)) return false;
    
    // Parse prefix
    int prefix = atoi(prefix_str);
    if (prefix < 0 || prefix > 32) return false;
    
    // Calcola range
    uint32_t netmask = create_netmask((uint8_t)prefix);
    uint32_t network = calculate_network_address(ip, netmask);
    uint32_t broadcast = calculate_broadcast_address(network, netmask);
    
    // Popola struct
    spec->type = SPEC_CIDR;
    spec->start_ip = network;
    spec->end_ip = broadcast;
    spec->prefix_len = (uint8_t)prefix;
    
    return true;
}

/**
 * Parse range: 10.0.0.1-10.0.0.50
 * 
 * Algoritmo:
 * 1. Split su '-'
 * 2. Parse start e end IP
 * 3. Valida start <= end
 */
bool parse_range(const char *input, IPSpec *spec) {
    char buf[64];
    strncpy(buf, input, sizeof(buf) - 1);
    buf[sizeof(buf) - 1] = '\0';
    
    // Tokenize: start-end
    char *start_str = strtok(buf, "-");
    char *end_str = strtok(NULL, "-");
    
    if (!start_str || !end_str) return false;
    
    // Parse IP
    uint32_t start, end;
    if (!string_to_ip(start_str, &start)) return false;
    if (!string_to_ip(end_str, &end)) return false;
    
    // Valida ordine
    if (start > end) return false;
    
    spec->type = SPEC_RANGE;
    spec->start_ip = start;
    spec->end_ip = end;
    spec->prefix_len = 0;  // Non usato
    
    return true;
}

/**
 * Parse single IP: 8.8.8.8
 */
bool parse_single(const char *input, IPSpec *spec) {
    uint32_t ip;
    if (!string_to_ip(input, &ip)) return false;
    
    spec->type = SPEC_SINGLE;
    spec->start_ip = ip;
    spec->end_ip = ip;  // start == end
    spec->prefix_len = 32;
    
    return true;
}

/**
 * Parse input (auto-detect formato)
 * 
 * Algoritmo:
 * 1. Cerca '/' → CIDR
 * 2. Cerca '-' → Range
 * 3. Altrimenti → Single
 */
bool parse_ip_spec(const char *input, IPSpec *spec) {
    if (!input || !spec) return false;
    
    if (strchr(input, '/')) {
        return parse_cidr(input, spec);
    } else if (strchr(input, '-')) {
        return parse_range(input, spec);
    } else {
        return parse_single(input, spec);
    }
}

/**
 * Espande specifica a set di IP
 * 
 * Algoritmo:
 * 1. Itera da start_ip a end_ip
 * 2. Aggiungi ogni IP al set
 * 
 * Nota: per /8 genera 16M IP! Usa con cautela.
 */
bool expand_spec_to_set(const IPSpec *spec, IPSet *set) {
    if (!spec || !set) return false;
    
    // Calcola size (overflow check)
    uint64_t count = (uint64_t)spec->end_ip - spec->start_ip + 1;
    
    // Limite sicurezza: max 1M IP
    if (count > 1000000) {
        return false;
    }
    
    // Espandi range
    for (uint32_t ip = spec->start_ip; ip <= spec->end_ip; ip++) {
        if (!ipset_add(set, ip)) {
            // Errore allocazione o duplicato
            // Continua comunque (duplicati OK)
        }
        
        // Protezione overflow: se ip == 0xFFFFFFFF, +1 fa wrap a 0
        if (ip == 0xFFFFFFFF) break;
    }
    
    return true;
}
EOF

echo "✅ parser.c implementato"

## ✅ Test Parser

In [ ]:
%%bash
cd /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/06

cat > test_parser.c << 'EOF'
#include <stdio.h>
#include "parser.h"
#include "ip_utils.h"

void print_spec(const IPSpec *spec) {
    char start[16], end[16];
    ip_to_string(spec->start_ip, start);
    ip_to_string(spec->end_ip, end);
    
    const char *type_str;
    switch (spec->type) {
        case SPEC_CIDR:   type_str = "CIDR"; break;
        case SPEC_RANGE:  type_str = "Range"; break;
        case SPEC_SINGLE: type_str = "Single"; break;
        default:          type_str = "Unknown"; break;
    }
    
    printf("  Type: %s\n", type_str);
    printf("  Start: %s\n", start);
    printf("  End: %s\n", end);
    if (spec->type == SPEC_CIDR) {
        printf("  Prefix: /%d\n", spec->prefix_len);
    }
}

void test_cidr() {
    printf("=== TEST parse_cidr() ===\n");
    
    const char *tests[] = {
        "192.168.1.0/24",
        "10.0.0.0/8",
        "172.16.0.0/12"
    };
    
    for (int i = 0; i < 3; i++) {
        printf("\nInput: %s\n", tests[i]);
        
        IPSpec spec;
        if (parse_ip_spec(tests[i], &spec)) {
            print_spec(&spec);
            
            uint32_t count = spec.end_ip - spec.start_ip + 1;
            printf("  Host: %u\n", count);
        } else {
            printf("  ✗ Parse failed\n");
        }
    }
    printf("\n");
}

void test_range() {
    printf("=== TEST parse_range() ===\n");
    
    const char *tests[] = {
        "10.0.0.1-10.0.0.50",
        "192.168.1.100-192.168.1.200",
        "8.8.8.8-8.8.8.10"
    };
    
    for (int i = 0; i < 3; i++) {
        printf("\nInput: %s\n", tests[i]);
        
        IPSpec spec;
        if (parse_ip_spec(tests[i], &spec)) {
            print_spec(&spec);
            
            uint32_t count = spec.end_ip - spec.start_ip + 1;
            printf("  Count: %u IP\n", count);
        } else {
            printf("  ✗ Parse failed\n");
        }
    }
    printf("\n");
}

void test_single() {
    printf("=== TEST parse_single() ===\n");
    
    const char *tests[] = {
        "8.8.8.8",
        "1.1.1.1",
        "192.168.1.1"
    };
    
    for (int i = 0; i < 3; i++) {
        printf("\nInput: %s\n", tests[i]);
        
        IPSpec spec;
        if (parse_ip_spec(tests[i], &spec)) {
            print_spec(&spec);
        } else {
            printf("  ✗ Parse failed\n");
        }
    }
    printf("\n");
}

void test_invalid() {
    printf("=== TEST Input Invalidi ===\n\n");
    
    const char *tests[] = {
        "192.168.1.0/33",      // Prefix > 32
        "10.0.0.50-10.0.0.1",  // start > end
        "999.999.999.999",     // IP invalido
        "192.168.1.0//24",     // Doppio delimitatore
    };
    
    for (int i = 0; i < 4; i++) {
        IPSpec spec;
        bool ok = parse_ip_spec(tests[i], &spec);
        printf("  %-25s → %s\n", tests[i], 
               ok ? "✗ Accettato (errore!)" : "✓ Rifiutato");
    }
    printf("\n");
}

void test_expansion() {
    printf("=== TEST expand_spec_to_set() ===\n\n");
    
    IPSpec spec;
    parse_ip_spec("192.168.1.0/28", &spec);  // /28 = 16 IP
    
    IPSet *set = ipset_create(32);
    
    printf("Espando 192.168.1.0/28...\n");
    expand_spec_to_set(&spec, set);
    
    printf("  IP generati: %zu\n\n", ipset_size(set));
    
    printf("Primi 5 IP:\n");
    for (size_t i = 0; i < 5 && i < ipset_size(set); i++) {
        char buf[16];
        ip_to_string(ipset_get(set, i), buf);
        printf("  [%zu] %s\n", i, buf);
    }
    
    printf("\nUltimi 5 IP:\n");
    size_t start = ipset_size(set) > 5 ? ipset_size(set) - 5 : 0;
    for (size_t i = start; i < ipset_size(set); i++) {
        char buf[16];
        ip_to_string(ipset_get(set, i), buf);
        printf("  [%zu] %s\n", i, buf);
    }
    
    ipset_free(set);
    printf("\n");
}

void test_auto_detect() {
    printf("=== TEST Auto-detect Formato ===\n\n");
    
    struct {
        const char *input;
        const char *expected;
    } tests[] = {
        {"192.168.1.0/24", "CIDR"},
        {"10.0.0.1-10.0.0.50", "Range"},
        {"8.8.8.8", "Single"},
    };
    
    for (int i = 0; i < 3; i++) {
        IPSpec spec;
        if (parse_ip_spec(tests[i].input, &spec)) {
            const char *type;
            switch (spec.type) {
                case SPEC_CIDR:   type = "CIDR"; break;
                case SPEC_RANGE:  type = "Range"; break;
                case SPEC_SINGLE: type = "Single"; break;
                default:          type = "Unknown"; break;
            }
            
            bool match = strcmp(type, tests[i].expected) == 0;
            printf("  %-25s → %s %s\n", 
                   tests[i].input, type,
                   match ? "✓" : "✗");
        }
    }
}

int main() {
    test_cidr();
    test_range();
    test_single();
    test_invalid();
    test_expansion();
    test_auto_detect();
    
    printf("\n✅ Tutti i test completati!\n");
    return 0;
}
EOF

gcc -std=c11 -Wall -Wextra -o test_parser \
    ip_utils.c ipset.c parser.c test_parser.c && ./test_parser

## 🔍 Analisi: strtok Gotchas

### ⚠️ strtok modifica la stringa!

```c
const char *input = "192.168.1.0/24";  // Stringa letterale
char *token = strtok(input, "/");     // ✗ CRASH!
                                        // (readonly memory)
```

**Soluzione: copia su stack**
```c
char buf[64];
strncpy(buf, input, sizeof(buf) - 1);
buf[sizeof(buf) - 1] = '\0';  // Termina sempre!

char *token = strtok(buf, "/");  // ✓ OK
```

### strtok è stateful

```c
// Prima chiamata: passa stringa
char *ip = strtok(buf, "/");

// Successive: passa NULL
char *prefix = strtok(NULL, "/");
              └─────┬────┘
           continua da dove ha lasciato
```

**⚠️ Non thread-safe!** Usa `strtok_r` per codice multithread.

## 🔍 Analisi: Overflow Protection

### Problema: Loop su uint32_t

```c
// SBAGLIATO per end_ip == 0xFFFFFFFF!
for (uint32_t ip = start; ip <= end; ip++) {
    // ...
}
// Se end = 0xFFFFFFFF:
//   ip = 0xFFFFFFFF
//   ip++  → 0x00000000  ← overflow!
//   ip <= 0xFFFFFFFF? → true
//   Loop infinito!
```

### ✅ Fix: Break esplicito

```c
for (uint32_t ip = start; ip <= end; ip++) {
    // ...
    
    if (ip == 0xFFFFFFFF) break;  // Ultimo IP possibile
}
```

### Alternativa: do-while

```c
uint32_t ip = start;
do {
    // Processa ip
} while (ip++ < end);  // Post-increment, poi check
```

## 📚 Recap

### ✅ Parser completo:

**Formati supportati:**
1. **CIDR**: `192.168.1.0/24`
2. **Range**: `10.0.0.1-10.0.0.50`
3. **Single**: `8.8.8.8`

**Funzioni:**
- `parse_ip_spec()` - Auto-detect formato
- `parse_cidr()` - Split `/`, calcola network/broadcast
- `parse_range()` - Split `-`, valida ordine
- `parse_single()` - Parse singolo IP
- `expand_spec_to_set()` - Range → IPSet

### 🎓 Concetti chiave:

- **strtok**: tokenizzazione, modifica stringa, stateful
- **strncpy**: copia sicura con size limit
- **Auto-detect**: `strchr` cerca delimitatori
- **Overflow**: break su `0xFFFFFFFF`

### 🔑 Pattern:

```c
// 1. Copia input (strtok safe)
char buf[64];
strncpy(buf, input, sizeof(buf) - 1);

// 2. Tokenize
char *tok1 = strtok(buf, delim);
char *tok2 = strtok(NULL, delim);

// 3. Parse tokens
uint32_t val1 = parse(tok1);
uint32_t val2 = parse(tok2);

// 4. Valida
if (val1 > MAX) return false;
```

### ➡️ Prossimi Notebook

**Notebook 7: Generator** - Generazione IP casuali  
**Notebook 8: CLI** - getopt, ProgramOptions  
**Notebook 9: Main** - Integrazione completa

---

## 🎉 Notebook 6 Completato!

**Funzioni**: 5 parser + espansione  
**Concetti**: strtok, string parsing, overflow protection

Continua con il Notebook 7! 🚀